Install needed libraries

In [26]:
%pip install requests
%pip install PyGithub
%pip install openai
%pip install --upgrade openai
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


  Attempting uninstall: openai
    Found existing installation: openai 1.34.0
    Uninstalling openai-1.34.0:
      Successfully uninstalled openai-1.34.0
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Mate\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Imports

In [95]:
from github import Github
from openai import OpenAI  
import json
import requests
import re
from dotenv import load_dotenv
import os

Function to check if input is already a GitHub repo

In [35]:
def is_github_repo_link(env):
    return env.startswith("https://github.com/") or env.startswith("http://github.com/")

Prompt for user

In [36]:
prompt = input("Enter a name of the environment or link to GitHub repo: ")
flag = True
if is_github_repo_link(prompt) == False:
   flag = False

Function for searching GitHub repositories

In [37]:
def retrieve_files(repo_urls):
    file_names = ['README.md', 'install.md', 'package.json']
    branches = ['main', 'master', 'develop']
    additional_paths = ['', 'docs/', 'src/']
    files_content = {file_name: [] for file_name in file_names}

    for url in repo_urls:
        if url.endswith('.git'):
            trimmed_url = url[:-4]
        else:
            trimmed_url = url

        file_found = {file_name: False for file_name in file_names}

        for branch in branches:
            for additional_path in additional_paths:
                for file_name in file_names:
                    if not file_found[file_name]:
                        file_url = f"{trimmed_url.rstrip('/')}/raw/{branch}/{additional_path}{file_name}"
                        try:
                            response = requests.get(file_url)
                            if response.status_code == 200:
                                files_content[file_name].append(response.text)
                                file_found[file_name] = True
                                print(f"Successfully retrieved {file_name} from {file_url}")
                            else:
                                print(f"Failed to retrieve {file_name} for {url} on branch {branch} at path {additional_path}. Status code: {response.status_code}")
                        except requests.exceptions.RequestException as e:
                            print(f"Error while trying to get {file_name} for {url} on branch {branch} at path {additional_path}: {e}")

    return files_content

OpenAI API for getting repositories

In [98]:
load_dotenv()
client = OpenAI(
    api_key = os.getenv("AZURE_OPENAI_KEY"),
    base_url=os.getenv("AZURE_OPENAI_ENDPOINT")
)
def generate_github_repos(prompt, model="gpt-3.5-turbo"):
    system_prompt = "You are an AI assistant that suggests relevant GitHub repositories based on a project description."
    user_prompt = f"Suggest GitHub repositories for a project that involves the following technologies: {prompt}. Provide only the URLs of the repositories."

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    suggested_repos = response.choices[0].message.content.strip()
    repo_urls = re.findall(r'(https?://github\.com/[^\s]+)', suggested_repos)

    return repo_urls

Analyze and pre-process the data

In [10]:
def parse_readme(content):
    sections = {}
    current_section = None
    lines = content.split('\n')

    for line in lines:
        header_match = re.match(r'^(#+)\s+(.*)', line)
        if header_match:
            level = len(header_match.group(1))
            section_name = header_match.group(2).strip()
            current_section = section_name
            sections[current_section] = []
        elif current_section:
            sections[current_section].append(line)

    for section in sections:
        sections[section] = "\n".join(sections[section])

    return sections

In [11]:
def parse_package_json(content):
    try:
        package_data = json.loads(content)
        return {
            "dependencies": package_data.get("dependencies", {}),
            "devDependencies": package_data.get("devDependencies", {}),
            "scripts": package_data.get("scripts", {})
        }
    except json.JSONDecodeError as e:
        print(f"Error parsing package.json: {e}")
        return {}

In [12]:
def collect_context(files_content):
    context = {}

    for file_name, contents in files_content.items():
        if file_name == 'README.md':
            for content in contents:
                context[file_name] = parse_readme(content)
        elif file_name == 'package.json':
            for content in contents:
                context[file_name] = parse_package_json(content)

    return context

Validation of output, this is just for us to see how is data being processed

In [14]:
#context is underlined beacuse it's defined below, in main code section, but that doesn't matter since this is jupyter notebook
if 'package.json' in context:
    package_info = context['package.json']
    dependencies = package_info.get('dependencies', {})
    dev_dependencies = package_info.get('devDependencies', {})
    scripts = package_info.get('scripts', {})

    print("Dependencies:", dependencies)
    print("Dev Dependencies:", dev_dependencies)
    print("Scripts:", scripts)
else:
    print("No 'package.json' found in context.")

NameError: name 'context' is not defined

In [15]:
#context is underlined beacuse it's defined below, in main code section, but that doesn't matter since this is jupyter notebook
if 'README.md' in context:
    readme_info = context['README.md']

    for section_name, section_content in readme_info.items():
        print(f"Section: {section_name}")
        print(section_content)
        print()
else:
    print("No 'README.md' found in context.")

NameError: name 'context' is not defined

Generate JSON file for Dev Container with OpenAI API

In [91]:
load_dotenv()
client = OpenAI(
    api_key = os.getenv("AZURE_OPENAI_KEY"),
    base_url=os.getenv("AZURE_OPENAI_ENDPOINT")
)
def generate_dev_container_json(context, model="gpt-3.5-turbo"):
    system_prompt = "You are an AI assistant that creates JSON file for dev container.\n\n"
    user_prompt = f"Generate JSON configuration for dev container based on the context provided: {context}"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    print("Raw API Response:")
    print(response)

    if 'data' in response:
        for message in response['data']:
            if message['role'] == 'assistant':
                generated_json = message['content']
                try:
                    parsed_json = json.loads(generated_json)
                    print("Generated Dev Container JSON Configuration:")
                    print(json.dumps(parsed_json, indent=4))
                    return parsed_json
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
                    return None
            else:
                print(f"Role '{message['role']}' not recognized.")
                return None
    else:
        print("No 'data' key found in API response.")
        return None


Main code

In [90]:
if flag == False:
  repos=generate_github_repos(prompt)
  print(repos)
  files=retrieve_files(repos)
else:
  repo=[prompt]
  files=retrieve_files(repo)
context = collect_context(files)
generate_dev_container_json(context)

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}